In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re
import numpy.ma as ma

from hbmep.config import Config
from hbmep.model.utils import Site as site

from models import NonHierarchicalBayesianModel
from constants import (
    DATA_PATH,
    TOML_PATH
)

In [2]:
df = pd.read_csv("/home/andres/data/hbmep-processed/L_CIRC/data.csv")
config = Config(toml_path=TOML_PATH)
model = NonHierarchicalBayesianModel(config=config)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [3]:
src = "/home/andres/repos/rat-mapping-paper/reports/L_CIRC/combine_response.pkl"
with open(src, "rb") as f:
    combinations = pickle.load(f)

In [4]:
set(k[1] for k in combinations.keys())


{'-C6LC',
 '-C6LE',
 '-C6LN',
 '-C6LNE',
 '-C6LNW',
 '-C6LS',
 '-C6LSE',
 '-C6LSW',
 '-C6LW',
 'C6LE-C6LC',
 'C6LE-C6LW',
 'C6LN-C6LC',
 'C6LNE-C6LC',
 'C6LNE-C6LSW',
 'C6LNW-C6LC',
 'C6LS-C6LC',
 'C6LS-C6LN',
 'C6LSE-C6LC',
 'C6LSE-C6LNW',
 'C6LSW-C6LC',
 'C6LW-C6LC'}

In [5]:
subjects = sorted(df['participant'].unique())
combos = ['Vertex', 'Radius', 'Diameter']
electrodes = ['C6LC', 'C6LE', 'C6LN', 'C6LNE', 'C6LNW', 'C6LS', 'C6LSE', 'C6LSW', 'C6LW']
muscles = model.response


In [ ]:
temp = ['C6LE-C6LW','C6LNE-C6LSW','C6LS-C6LN','C6LSE-C6LNW']
ord = []
arr = []
mask = []
for sub in subjects:
    for combo in combos:
        for elec in electrodes:
            for m in muscles:
                if combo == 'Vertex':
                    position = '-'+elec
                if combo == 'Radius':
                    position = elec + '-' + 'C6LC'
                if combo == 'Diameter':
                    for e in electrodes:
                        if elec != e:
                            temppos = elec + '-' + e
                            if temppos in temp:
                                position = temppos
                key = (sub, position, m)
                
                if key in combination.keys():
                    tcomb = combination[key]['a']
                    tcomb = tcomb.reshape(-1, 1)
                    arr.append(tcomb)
                    mask.append(np.full((4000,1), False))
                else:
                    arr.append(np.zeros(4000).reshape(-1, 1))
                    mask.append(np.full((4000, 1), True))
                    
arr = np.array(arr)
arr = arr.reshape(len(subjects), len(combos),len(electrodes), len(muscles), *arr.shape[1:])
mask = np.array(mask)
mask = mask.reshape(len(subjects), len(combos),len(electrodes),len(muscles), *mask.shape[1:])

arr = ma.masked_array(arr, mask=mask)
arr = arr[..., 0]
                

In [7]:
combinations_ordered  = df[model.features].apply(tuple, axis=1).unique().tolist()
len(combinations_ordered)

168

In [8]:
y = []

for c in combinations_ordered:
    for response in model.response:
        combination = (*c, response)
        y_curr = combinations[combination]['response']
        y.append(y_curr)

y = np.array(y)
y = y.reshape(len(combinations_ordered), len(model.response), -1)
y.shape
    

(168, 6, 1000)

In [12]:
y_max = []

for c in combinations_ordered:
    subject = c[0]
    ind = [i for i, combination in enumerate(combinations_ordered) if combination[0] == subject] #21
    curr_y = y[ind, ...]
    curr_y_max = np.max(curr_y, axis=(0, -1), keepdims=True) #6
    y_max.append(curr_y_max) 

y_max = np.array(y_max)
y_max = y_max[:, 0, ...]
y_max.shape
# y_max.reshape(len(combinations_ordered), -1)

# y_max = y_max[..., None]
# y_max.shape


(168, 6, 1)

In [11]:
y_max.shape

(168, 1, 6, 1)

In [27]:
y = y / y_max
y.max()


1.0

In [17]:
combinations.keys()

dict_keys([('amap01', 'C6LSW-C6LC', 'LADM'), ('amap01', 'C6LSW-C6LC', 'LBiceps'), ('amap01', 'C6LSW-C6LC', 'LDeltoid'), ('amap01', 'C6LSW-C6LC', 'LECR'), ('amap01', 'C6LSW-C6LC', 'LFCR'), ('amap01', 'C6LSW-C6LC', 'LTriceps'), ('amap01', '-C6LSE', 'LADM'), ('amap01', '-C6LSE', 'LBiceps'), ('amap01', '-C6LSE', 'LDeltoid'), ('amap01', '-C6LSE', 'LECR'), ('amap01', '-C6LSE', 'LFCR'), ('amap01', '-C6LSE', 'LTriceps'), ('amap01', 'C6LS-C6LC', 'LADM'), ('amap01', 'C6LS-C6LC', 'LBiceps'), ('amap01', 'C6LS-C6LC', 'LDeltoid'), ('amap01', 'C6LS-C6LC', 'LECR'), ('amap01', 'C6LS-C6LC', 'LFCR'), ('amap01', 'C6LS-C6LC', 'LTriceps'), ('amap01', 'C6LSE-C6LC', 'LADM'), ('amap01', 'C6LSE-C6LC', 'LBiceps'), ('amap01', 'C6LSE-C6LC', 'LDeltoid'), ('amap01', 'C6LSE-C6LC', 'LECR'), ('amap01', 'C6LSE-C6LC', 'LFCR'), ('amap01', 'C6LSE-C6LC', 'LTriceps'), ('amap01', 'C6LSE-C6LNW', 'LADM'), ('amap01', 'C6LSE-C6LNW', 'LBiceps'), ('amap01', 'C6LSE-C6LNW', 'LDeltoid'), ('amap01', 'C6LSE-C6LNW', 'LECR'), ('amap01', '

In [6]:
combination = pickle.load(open("/home/andres/repos/rat-mapping-paper/reports/L_SHIE/combine.pkl", "rb"))

In [8]:
df['compound_position'].unique()

array(['C6LX-C6LC', 'C6LC-', 'C6LC-C6LX', '-C6LC'], dtype=object)